In [26]:
import config

from imageai.Detection import ObjectDetection
import os
import cv2
import numpy as np
import shutil

from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, UpSampling2D
from keras import backend as K

import random
import glob
# import wandb
# from wandb.keras import WandbCallback
import subprocess
import os
from PIL import Image
import numpy as np
from matplotlib.pyplot import imshow, figure

from keras.layers import Lambda, Reshape, Permute, Input, add, Conv3D, GaussianNoise, concatenate
from keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Add
from keras.models import Model

/home/pratt3000/Documents/projects/Violence Detection-v3/models/yolo-tiny.h5


In [27]:
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3() 
detector.setModelPath(config.PRETRAINED_MODEL_PATH)
detector.loadModel(detection_speed = config.OBJECT_DETECTION_SPEED) #change parameter to adjust accuracy and speed
custom = detector.CustomObjects(person=True)

In [28]:
hyperparams = {
    "num_epochs": 10, 
    "batch_size": 32,
    "FONT" : cv2.FONT_HERSHEY_SIMPLEX,
    "IMG_SIZE" : 64,
    "FRAME_BATCH_SIZE" : 4,
    "THRESHOLD_DIFF_TOLERANCE" : 35  
    }

In [29]:
# wandb.init(config=hyperparams)
# config = wandb.config

In [36]:
def my_generater(batch_size):     
  ########### YET TO ADD VALIDATION/TEST/OTHER DATASETS ############
    list_fight=os.listdir("/home/pratt3000/Documents/projects/Violence Detection-v3/movies/Violence")
    list_no_fight=os.listdir("/home/pratt3000/Documents/projects/Violence Detection-v3/movies/NonViolence")

    fight_final=random.sample(list_fight, 800)
    no_fight_final=random.sample(list_no_fight,800)

    fight_labels = []
    no_fight_labels = []

    for i in range (800):
        fight_labels.append([1,0])
        no_fight_labels.append([0,1])

    final = fight_final + no_fight_final
    labels = fight_labels + no_fight_labels

    c = list(zip(final,labels))
    random.shuffle(c)
    names, labels = zip(*c)

    images_batches=[]
    labelsss=[]

    for i in range(len(names)): ##no. of videos loop
       
        if labels[i]==[0,1]:
            in_dir= "/home/pratt3000/Documents/projects/Violence Detection-v3/movies/NonViolence"
        else:
            in_dir= "/home/pratt3000/Documents/projects/Violence Detection-v3/movies/Violence"
        in_file = os.path.join(in_dir, names[i])
        vidcap = cv2.VideoCapture(in_file)
        length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        for j in range(int(length/config.FRAME_BATCH_SIZE)):
            detections_temp_2=[]
            success,frame_temp = vidcap.read()
            frame_temp = cv2.cvtColor(frame_temp, cv2.COLOR_BGR2GRAY)
            detections_temp = get_boxes(frame_temp) 
            frame_temp = mask_frame(frame, detection_temp, detections_temp_2)                    
            images_frame_batches=[]
           
            for k in range(config.FRAME_BATCH_SIZE - 1):
               
                success,frame = vidcap.read()
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   
                detections = get_boxes(frame)                   
                frame = mask_frame(frame, detections, detections_temp)          
                diff = get_frame_difference(frame, frame_temp)
                images_frame_batches.append(diff)
                frame_temp = frame
                detections_temp=detections
           
            images_batches.append(images_frame_batches)
        
        aux = np.ones([int(length/config.FRAME_BATCH_SIZE),2])
        labelss = labels[i]*aux
        labelss = labelss.tolist()
        lebelsss.append(labelss)
    
    labelsss_final=np.reshape(labelsss,(len(names)*int(length/config.FRAME_BATCH_SIZE),2))
    counter = 0
    input_images=[]
    output_labels
    while True:
        input_imgs=np.zeros(
            (batch_size,config.FRAME_BATCH_SIZE, config.IMG_SIZE, config.IMG_SIZE, 1))
        if (counter+batch_size >= len(labelsss)):
            counter = 0
       
        for i in range(batch_size):
            input_imgs = images_batches[counter + i]
            input_images.append(input_imgs)
            output_label = labelsss_final[counter + i]
            output_labels.append(output_label)
            input_imgs[i] /= 255.
            
        yield (input_images, output_labels)
        counter += batch_size

In [31]:
def get_boxes(frame):
    _, detections = detector.detectCustomObjectsFromImage(
        custom_objects=custom,
        input_type="array",
        input_image= frame,
        output_type="array"
        )
    return detections

def frame_preprocessing(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, dsize=(config.IMG_SIZE, config.IMG_SIZE), interpolation=cv2.INTER_CUBIC)

    return frame

def get_frame_difference(frame_1, frame_2):
    frame = cv2.absdiff(frame_1, frame_2)
    ret,thresh1 = cv2.threshold(frame,config.THRESHOLD_DIFF_TOLERANCE,255,cv2.THRESH_BINARY)   #127 is threshold
    return thresh1

def mask_frame(frame, detections, detections_temp):
    mask = np.zeros(frame.shape, dtype=np.uint8)

    for eachObject in detections:
        x1,y1,x2,y2 = eachObject["box_points"]
        mask = cv2.rectangle(mask, (x1, y1),(x2,y2), (255,255,255), -1) 

    for eachObject in detections_temp:
        x1,y1,x2,y2 = eachObject["box_points"]
        mask = cv2.rectangle(mask, (x1, y1),(x2,y2), (255,255,255), -1) 

    result = cv2.bitwise_and(frame, mask)   # Mask input image with binary mask
    result[mask==0] = 255   # Optional : set background -> now white/ by default black

    return result

In [32]:



inp = Input((config.FRAME_BATCH_SIZE - 1,config.IMG_SIZE,config.IMG_SIZE,1))
permuted = Permute((2,3,4,1))(inp)
noise = GaussianNoise(0.1)(permuted)
c=4
x = Permute((4,1,2,3))(noise)
x =(ConvLSTM2D(filters=c, kernel_size=(3,3),padding='same',name='conv_lstm1', return_sequences=True))(x)

c1=(BatchNormalization())(x)
x = Dropout(0.2)(x)
x =(TimeDistributed(MaxPooling2D(pool_size=(2,2))))(c1)

x =(ConvLSTM2D(filters=2*c,kernel_size=(3,3),padding='same',name='conv_lstm2',return_sequences=True))(x)
c2=(BatchNormalization())(x)
x = Dropout(0.2)(x)

x =(TimeDistributed(MaxPooling2D(pool_size=(2,2))))(c2)
x =(ConvLSTM2D(filters=4*c,kernel_size=(3,3),padding='same',name='conv_lstm3',return_sequences=True))(x)

x =(TimeDistributed(UpSampling2D(size=(2, 2))))(x)
x =(ConvLSTM2D(filters=4*c,kernel_size=(3,3),padding='same',name='conv_lstm4',return_sequences=True))(x)
x =(BatchNormalization())(x)

x =(ConvLSTM2D(filters=2*c,kernel_size=(3,3),padding='same',name='conv_lstm5',return_sequences=True))(x)
x =(BatchNormalization())(x)
x = Add()([c2, x])
x = Dropout(0.2)(x)

x =(TimeDistributed(UpSampling2D(size=(2, 2))))(x)
x =(ConvLSTM2D(filters=c,kernel_size=(3,3),padding='same',name='conv_lstm6',return_sequences=False))(x)
x =(BatchNormalization())(x)

x = (Flatten())(x)
x = (Dense(units=50, activation='relu'))(x)

x = (Dense(units=2, activation='relu'))(x)

model=Model(inputs=[inp], outputs=[x])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 3, 512, 512,  0                                            
__________________________________________________________________________________________________
permute_3 (Permute)             (None, 512, 512, 1,  0           input_8[0][0]                    
__________________________________________________________________________________________________
gaussian_noise_2 (GaussianNoise (None, 512, 512, 1,  0           permute_3[0][0]                  
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 3, 512, 512,  0           gaussian_noise_2[0][0]           
__________________________________________________________________________________________________
conv_lstm1

In [42]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

train
model.fit_generator(
                    my_generater(4, train_dir),
                    steps_per_epoch=steps_per_epoch//4,
                    epochs=10, 
                    callbacks=[ImageCallback(), WandbCallback()],
                    validation_steps=validation_steps//4,
                    validation_data=my_generator(4, val_dir)
                    )

NameError: name &#39;train_dir&#39; is not defined

In [ ]:
class ImageCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        validation_X, validation_y = next(
            my_generator(15, val_dir))
        output = self.model.predict(validation_X)
        wandb.log({
            "input": [wandb.Image(np.concatenate(np.split(c, 5, axis=2), axis=1)) for c in validation_X],
            "output": [wandb.Image(np.concatenate([validation_y[i], o], axis=1)) for i, o in enumerate(output)]
        }, commit=False)

In [11]:
import config
from keras.layers import Lambda, Reshape, Permute, Input, add, Conv3D, GaussianNoise, concatenate
from keras.callbacks import Callback
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten, UpSampling2D
from keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Add
from keras.models import Model
import config

from imageai.Detection import ObjectDetection
import os
import cv2
import numpy as np
import main 


model = main.get_model()
#model.load_weights("/home/pratt3000/Documents/projects/Violence Detection-v3/models/classification/saved-model-01-0.56.hdf5")
model.summary()

error: OpenCV(4.4.0) /tmp/pip-req-build-6amqbhlx/opencv/modules/imgproc/src/resize.cpp:3929: error: (-215:Assertion failed) !ssize.empty() in function &#39;resize&#39;
